# IUM - semestr 24L
## Zadanie 4
### Zespół 10, w składzie: 
- Małgorzata Kozłowska (318681), 
- Kacper Murygin (#TODO)
### Prowadzący: Paweł Zawistowski


### Treść zadania:
> “Mamy pomysł na świetne narzędzie dla naszych użytkowników – zacznijmy generować listę top utworów, ale z wyprzedzeniem tygodniowym! Będziemy szybsi od konkurencji i zwiększymy zaangażowanie.”

Do wykonania zadania otrzymaliśmy dostęp do danych ze zbiorów:
- artists.jsonl
- sessions.jsonl
- track_storage.jsonl
- tracks.jsonl
- users.jsonl

### Ogólna analiza powyższych zbiorów:
#### artists.jsonl
Zbiór danych zawiera informacje o 'id' - identyfikatorze danego artysty, o 'name' - jego imieniu scenicznym oraz o 'genres' - gatunkku/liście gatunków, do jakich jego twórczość należy.

#### sessions.jsonl
Zbiór danych zawiera informacje o sesjach użytkowników. Do zarejestrowanych danych należy 'session_id' - id sesji, 'timestamp' - czas wykonania zdarzenia, 'user_id' - unikalny identyfikator użytkownika, 'track_id' - unikalny identyfikator utworu oraz 'event_type', mówiący o typu zdarzenia, jakie zostało zanotowane.

#### track_storage.jsonl


#### tracks.jsonl

#### users.jsonl

### Analiza zbiorów z perspektywy realizacji zadania

#### artists.jsonl - otrzymany
#### sessions.jsonl
#### track_storage.jsonl
#### tracks.jsonl
#### users.jsonl

TODO:
- braki w zbiorach
- z jakich będziemy korzystać
- jakie pola


Podejścia
Dwa modele:
- dla starych hitów (nowy serial, soundtrack): po 'release_date' i 'session_id' ('popularity')- wzrost odsłuchiwań - Oldies but Goodies
- dla genres/wszystkich - filtrujemy względem 'genres' : pola 'popularity'
